# Output just data for ice crystal calculations

### For transport to Cheyenne because of technical issues on NIRD

In [22]:
import sys
# Add common resources folder to path
sys.path.append("/mnt/mcc-ns9600k/jonahks/git_repos/netcdf_analysis/Common/")
sys.path.append("/mnt/mcc-ns9600k/jonahks/git_repos/netcdf_analysis/")
sys.path.append("/home/jonahks/git_repos/netcdf_analysis/")
sys.path.append("/home/jonahks/git_repos/netcdf_analysis/Common/")

from imports import (
    pd, np, xr, mpl, plt, sns, os, 
    datetime, sys, crt, gridspec,
    ccrs, metrics, Iterable, re
    )

from functions import (
    masked_average, add_weights, sp_map,
    season_mean, get_dpm, leap_year, share_ylims,
    to_png, average_and_wrap, mute_ax
    )

from classes import SatComp_Metric, CT_SLF_Metric
from collections import deque
%matplotlib inline

import dask
import glob

### Check running location and adjust working directory appropriately.

In [2]:
host = os.uname()[1]
if 'jupyter' in host.split('-'): # Check if running on NIRD through the Jupyter Hub
    print('Running through MC2 Jupyter Hub')
    model_dir = '/mnt/mcc-ns9600k/jonahks/'
    os.chdir(model_dir)

else:  # Assume that we're running on a local machine and mounting NIRD
    print('Running on %s, attempting to mount ns9600k/jonahks/ from NIRD' % str(host))
    os.system('fusermount -zu ~/drivemount/')  # unmount first
    os.system('sshfs jonahks@login.nird.sigma2.no:"p/jonahks/" ~/drivemount/')    # Calling mountnird from .bashrc doesn't work
    os.chdir('/home/jonahks/drivemount/')
    save_dir = '~/DATAOUT/'
    save_to = os.path.expanduser(save_dir)

output_dir = 'figures/'
case_dir = 'satcomp/'   # inconsistent label compared to jupy_test
fb_dir ='feedbacks_soden/mod_by_me/output_jonahks/'

# Check that each important directory can be accessed:    
access_paths = os.path.exists(output_dir) and os.path.exists(case_dir) and os.path.exists(fb_dir)
print('Can access all directory paths:', access_paths)

Running through MC2 Jupyter Hub
Can access all directory paths: True


## Load data into metric classes

In [3]:
allcases = SatComp_Metric(case_dir)

allcases.add_case('20200504_145018_fitting_runs_cam6satcomp_wbf_1_inp_1', label="CAM6-Oslo")
allcases.add_case('CESM2_slfvars', label="CAM6")
allcases.add_case('20200414_205148_singleparam_cam61satcomp_wbf_1_inp_1', label="CAM6-OsloIce")

allcases.add_case("20200512_013308_fitting_runs_cam6satcomp16_wbf_1.25_inp_10",label='CAM6-Oslo \n Fit 1')
# allcases.add_case('20200414_205148_singleparam_cam61satcomp_wbf_1_inp_0.05',label='CAM6-OsloIce \n Fit 2')
allcases.add_case('20200629_morn_cam61satcomp_wbf_0.5_inp_0.05',label='CAM6-OsloIce \n Fit 2')
allcases.add_case('20200512_012745_fitting_runs_cam61satcomp_wbf_0.2_inp_0.1',label='CAM6-OsloIce \n Fit 3')
allcases.add_case('20200713_CESM2_satcomp_wbf_1_inp_100',label="CAM6 \n Fit 4")

Loading GOCCP data...done.
Loading CALIOP SLFs...done
Loading CERES-EBAF fluxes...done.
Trying to load concatenated file for 20200504_145018_fitting_runs_cam6satcomp_wbf_1_inp_1
20200504_145018_fitting_runs_cam6satcomp_wbf_1_inp_1 load successfully.
Trying to load concatenated file for CESM2_slfvars
CESM2_slfvars load successfully.
Trying to load concatenated file for 20200414_205148_singleparam_cam61satcomp_wbf_1_inp_1
20200414_205148_singleparam_cam61satcomp_wbf_1_inp_1 load successfully.
Trying to load concatenated file for 20200512_013308_fitting_runs_cam6satcomp16_wbf_1.25_inp_10
20200512_013308_fitting_runs_cam6satcomp16_wbf_1.25_inp_10 load successfully.
Trying to load concatenated file for 20200629_morn_cam61satcomp_wbf_0.5_inp_0.05
20200629_morn_cam61satcomp_wbf_0.5_inp_0.05 load successfully.
Trying to load concatenated file for 20200512_012745_fitting_runs_cam61satcomp_wbf_0.2_inp_0.1
20200512_012745_fitting_runs_cam61satcomp_wbf_0.2_inp_0.1 load successfully.
Trying to load

### Extract the desired variables from the concatenated file and the first model output (for hyam and hybm)

In [44]:
for case_label in allcases.cases:
    print(case_label)
    
    ex_case = allcases.get_case(case_label)
    ex_da = ex_case.case_da

    file_list0 = glob.glob('%s/%s/*.nc' % (ex_case.case_dir,ex_case.case))
    file_list = glob.glob('%s/%s/atm/hist/*.nc' % (ex_case.case_dir,ex_case.case))
    
    file_list0.sort()
    file_list.sort()
    print(file_list0[0])
    print(file_list[0])
    _hy_file = xr.open_dataset(file_list[0])

    _levvars = _hy_file[['hyai','hybi','hyam','hybm']]
    _icevars = ex_da[['FREQI','AWNI','PS','CLDICE']]

    _allvars = xr.merge([_levvars,_icevars])

20200504_145018_fitting_runs_cam6satcomp_wbf_1_inp_1
satcomp//20200504_145018_fitting_runs_cam6satcomp_wbf_1_inp_1/20200504_145018_fitting_runs_cam6satcomp_wbf_1_inp_1.nc
satcomp//20200504_145018_fitting_runs_cam6satcomp_wbf_1_inp_1/atm/hist/20200504_145018_fitting_runs_cam6satcomp_wbf_1_inp_1.cam.h0.2009-03.nc
CESM2_slfvars
satcomp//CESM2_slfvars/CESM2_slfvars.nc
satcomp//CESM2_slfvars/atm/hist/CESM2_slfvars.cam.h0.2009-03.nc
20200414_205148_singleparam_cam61satcomp_wbf_1_inp_1
satcomp//20200414_205148_singleparam_cam61satcomp_wbf_1_inp_1/20200414_205148_singleparam_cam61satcomp_wbf_1_inp_1.nc
satcomp//20200414_205148_singleparam_cam61satcomp_wbf_1_inp_1/atm/hist/20200414_205148_singleparam_cam61satcomp_wbf_1_inp_1.cam.h0.2009-03.nc
20200512_013308_fitting_runs_cam6satcomp16_wbf_1.25_inp_10
satcomp//20200512_013308_fitting_runs_cam6satcomp16_wbf_1.25_inp_10/20200512_013308_fitting_runs_cam6satcomp16_wbf_1.25_inp_10.nc
satcomp//20200512_013308_fitting_runs_cam6satcomp16_wbf_1.25_inp_10

In [42]:
_allvars.to_netcdf('/mnt/mcc-ns9600k/jonahks/inp_obs/examplelabel.nc')

PermissionError: [Errno 13] Permission denied: b'/mnt/mcc-ns9600k/jonahks/inp_obs/examplelabel.nc'

In [40]:
os.getcwd()

'/mnt/mcc-ns9600k/jonahks'

In [38]:
os.listdir('inp_obs')

['inp_obs.csv']

In [9]:
ex_case = allcases.get_case('20200504_145018_fitting_runs_cam6satcomp_wbf_1_inp_1')

ex_da = ex_case.case_da

In [11]:
ex_case.case_dir

'satcomp/'

In [12]:
ex_case.case

'20200504_145018_fitting_runs_cam6satcomp_wbf_1_inp_1'

In [26]:
file_list = glob.glob('%s/%s/atm/hist/*.nc' % (ex_case.case_dir,ex_case.case))

In [27]:
file_list.sort()

In [33]:
_hy_file = xr.open_dataset(file_list[0])

_levvars = _hy_file[['hyai','hybi','hyam','hybm']]

_icevars = ex_da[['FREQI','AWNI','PS','CLDICE']]

_allvars = xr.merge([_levvars,_icevars])

In [34]:
_allvars

<xarray.Dataset>
Dimensions:  (ilev: 33, lat: 96, lev: 32, lon: 144, time: 48)
Coordinates:
  * ilev     (ilev) float64 2.255 5.032 10.16 18.56 ... 947.4 967.5 985.1 1e+03
  * lev      (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
  * lat      (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 84.32 86.21 88.11 90.0
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 2009-06-01 2009-07-01 ... 2013-05-01
Data variables:
    hyai     (ilev) float64 ...
    hybi     (ilev) float64 ...
    hyam     (lev) float64 ...
    hybm     (lev) float64 ...
    FREQI    (time, lev, lat, lon) float32 ...
    AWNI     (time, lev, lat, lon) float32 ...
    PS       (time, lat, lon) float32 ...
    CLDICE   (time, lev, lat, lon) float32 ...

In [6]:
ice_conc = ex_da['AWNI'] / ex_da['FREQI']

avg_conc = ice_conc.mean(dim=['time','lon'])*1e-3

colorsf = plt.cm.Reds(np.linspace(0.0,1.0,9))

/opt/conda/lib/python3.7/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [5]:
allcases